In [1]:
import numpy as np
import matplotlib.pyplot as plt

from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class
from Druckrohrleitung.Druckrohrleitung_class_file import Druckrohrleitung_class

In [2]:
#define constants

# physics
g       = 9.81                          # gravitational acceleration [m/s²]
rho     = 1000.                         # density of water [kg/m³]

# pipeline
L       = 1000.                         # length of pipeline [m]
D       = 1.                            # pipe diameter [m]
A_pipe  = D**2/4*np.pi                  # pipeline area
h_pipe  = 200                           # hydraulic head without reservoir [m] 
alpha   = np.arcsin(h_pipe/L)           # Höhenwinkel der Druckrohrleitung 
n       = 10                           # number of pipe segments in discretization
#consider replacing Q0 with a vector be be more flexible in initial conditions
Q0      = 2.                            # initial flow in whole pipe [m³/s]
v0      = Q0/A_pipe                     # initial flow velocity [m/s]
f_D     = 0.1                           # Darcy friction factor
c       = 400.                          # propagation velocity of the pressure wave [m/s]
#consider prescribing a total simulation time and deducting the number of timesteps from that
nt      = 100                           # number of time steps after initial conditions

# derivatives of the pipeline constants
dx      = L/n                           # length of each pipe segment
dt      = dx/c                          # timestep according to method of characterisitics
nn      = n+1                           # number of nodes
h_res   = 20.                           # water level in upstream reservoir [m]
p0      = rho*g*h_res-v0**2*rho/2
pl_vec  = np.arange(0,nn*dx,dx)         # pl = pipe-length. position of the nodes on the pipeline
t_vec   = np.arange(0,nt*dt,dt)         # time vector
h_vec   = np.arange(0,n+1)*h_pipe/n     # hydraulic head of pipeline at each node  np.arange(0,0) does not yield the intended result
v_init  = np.full(nn,Q0/(D**2/4*np.pi)) # initial velocity distribution in pipeline
p_init  = (rho*g*(h_res+h_vec)-v_init**2*rho/2)-(f_D*pl_vec/D*rho/2*v_init**2) # ref Wikipedia: Darcy Weisbach


# reservoir
initial_level               = h_res     # water level in upstream reservoir [m]
# replace influx by vector
initial_influx              = 0.        # initial influx  of volume to the reservoir [m³/s]
initial_outflux             = Q0        # initial outflux of volume from the reservoir to the pipeline [m³/s]
initial_pipeline_pressure   = p0        # Initial condition for the static pipeline pressure at the reservoir (= hydrostatic pressure - dynamic pressure) 
initial_pressure_unit       = 'Pa'      # DO NOT CHANGE! for pressure conversion in print statements and plot labels 
conversion_pressure_unit    = 'mWS'      # for pressure conversion in print statements and plot labels
area_base                   = 20.       # total base are of the cuboid reservoir [m²]   
area_outflux                = A_pipe    # outlfux area of the reservoir, given by pipeline area [m²]
critical_level_low          = 0.        # for yet-to-be-implemented warnings[m]
critical_level_high         = np.inf    # for yet-to-be-implemented warnings[m]

# make sure e-RK4 method of reservoir has a small enough timestep to avoid runaway numerical error
nt_eRK4                     = 1000      # number of simulation steps of reservoir in between timesteps of pipeline              
simulation_timestep         = dt/nt_eRK4



#### Ideas for checks after constant definitions: 

- Check that the initial pressure is not negative:
    - may happen, if there is too little hydraulic head to create the initial flow conditions with the given friction
<br>
<br>
- plausbility checks?
    - area > area_outflux ?
    - propable ranges for parameters?
    - angle and height/length fit together?
    

In [3]:
# create objects

V = Ausgleichsbecken_class(area_base,area_outflux,critical_level_low,critical_level_high,simulation_timestep)
V.set_initial_level(initial_level) 
V.set_influx(initial_influx)
V.set_outflux(initial_outflux)
V.set_pressure(initial_pipeline_pressure,initial_pressure_unit,conversion_pressure_unit)

pipe = Druckrohrleitung_class(L,D,n,alpha,f_D)
pipe.set_pressure_propagation_velocity(c)
pipe.set_number_of_timesteps(nt)
pipe.set_initial_pressure(p_init,initial_pressure_unit,conversion_pressure_unit)
pipe.set_initial_flow_velocity(v_init)

# display the attributes of the created reservoir and pipeline object
# V.get_info(full=True)
# pipe.get_info()

The cuboid reservoir has the following attributes: 
----------------------------- 
Base area             =       20.0       m² 
Outflux area          =       0.785      m² 
Current level         =       20.0       m
Critical level low    =       0.0        m 
Critical level high   =       inf        m 
Volume in reservoir   =       400.0      m³ 
Current influx        =       0.0        m³/s 
Current outflux       =       2.0        m³/s 
Current pipe pressure =       1447.306   Torr 
Simulation timestep   =       0.00025    s 
----------------------------- 

The pipeline has the following attributes: 
----------------------------- 
Length                =       1000.0     m 
Diameter              =       1.0        m 
Number of segments    =       10         
Number of nodes       =       11         
Length per segments   =       100.0      m 
Pipeline angle        =       0.201      rad 
Pipeline angle        =       11.537° 
Darcy friction factor =       0.1        
Density of liqui

In [4]:
# initialization for timeloop

# prepare the vectors in which the pressure and velocity distribution in the pipeline from the previous timestep are stored
v_old = v_init.copy()
p_old = p_init.copy()

# prepare the vectors in which the temporal evolution of the boundary conditions are stored
    # keep in mind, that the velocity at the turbine and the pressure at the reservoir are set manually and
        # through the time evolution of the reservoir respectively  
    # the pressure at the turbine and the velocity at the reservoir are calculated from the method of characteristics
v_boundary_res  = np.empty_like(t_vec)
v_boundary_tur  = np.empty_like(t_vec)
p_boundary_res  = np.empty_like(t_vec)
p_boundary_tur  = np.empty_like(t_vec)

# prepare the vectors that store the temporal evolution of the level in the reservoir
level_vec       = np.full_like(t_vec,initial_level)  # level at the end of each pipeline timestep
level_vec_2     = np.empty([nt_eRK4])   # level throughout each reservoir timestep-used for plotting and overwritten afterwards

# set the boudary conditions for the first timestep
v_boundary_res[0]       = v_old[0]
v_boundary_tur[0]       = v_old[-1] 
v_boundary_tur[1:]      = 0                                     # instantaneous closing
# v_boundary_tur[0:20]    = np.linspace(v_old[-1],0,20)    # overwrite for finite closing time - linear case
const = int(np.min([100,round(nt/1.1)]))
v_boundary_tur[0:const]    = v_old[1]*np.cos(t_vec[0:const]*2*np.pi/5)**2
p_boundary_res[0]       = p_old[0]
p_boundary_tur[0]       = p_old[-1]



In [5]:
%matplotlib qt5
# time loop

# create a figure and subplots to display the velocity and pressure distribution across the pipeline in each pipeline step
fig1,axs1 = plt.subplots(2,1)
axs1[0].set_title('Pressure distribution in pipeline')
axs1[1].set_title('Velocity distribution in pipeline')
axs1[0].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[0].set_ylabel(r'$p$ ['+conversion_pressure_unit+']')
axs1[1].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[1].set_ylabel(r'$v$ [$\mathrm{m} / \mathrm{s}$]')
lo_00, = axs1[0].plot(pl_vec,pressure_conversion(pipe.p_old,initial_pressure_unit, conversion_pressure_unit)[0],marker='.')
lo_01, = axs1[1].plot(pl_vec,pipe.v_old,marker='.')
axs1[0].autoscale()
axs1[1].autoscale()
# displaying the reservoir level within each pipeline timestep
# axs1[2].set_title('Level reservoir')
# axs1[2].set_xlabel(r'$t$ [$\mathrm{s}$]')
# axs1[2].set_ylabel(r'$h$ [m]')
# lo_02, = axs1[2].plot(level_vec_2)
# axs1[2].autoscale()
fig1.tight_layout()
plt.show()
plt.pause(1)

# loop through time steps of the pipeline
for it_pipe in range(1,pipe.nt):

# for each pipeline timestep, execute nt_eRK4 timesteps of the reservoir code
    # set initial conditions for the reservoir time evolution calculted with e-RK4
    V.pressure  = p_old[0]
    V.outflux   = v_old[0]
    # calculate the time evolution of the reservoir level within each pipeline timestep to avoid runaway numerical error
    for it_res in range(nt_eRK4):
        V.e_RK_4()                                                              # call e-RK4 to update outflux
        V.level = V.update_level(V.timestep)                                    # 
        V.set_volume()                                                          # update volume in reservoir
        level_vec_2[it_res] = V.level                                           # save for plotting
        if (V.level < critical_level_low) or (V.level > critical_level_high):   # make sure to never exceed critical levels
            i_max = it_pipe                                                     # for plotting only calculated values
            break                                                               
    level_vec[it_pipe] = V.level                                                  

    # set boundary conditions for the next timestep of the characteristic method
    p_boundary_res[it_pipe] = rho*g*V.level-v_old[1]**2*rho/2
    v_boundary_res[it_pipe] = v_old[1]+1/(rho*c)*(p_boundary_res[it_pipe]-p_old[1])-f_D*dt/(2*D)*abs(v_old[1])*v_old[1] \
        +dt*g*np.sin(alpha)

    # the the boundary conditions in the pipe.object and thereby calculate boundary pressure at turbine
    pipe.set_boundary_conditions_next_timestep(v_boundary_res[it_pipe],p_boundary_res[it_pipe],v_boundary_tur[it_pipe])
    p_boundary_tur[it_pipe] = pipe.p_boundary_tur

    # perform the next timestep via the characteristic method
    pipe.timestep_characteristic_method()

    # plot some stuff
        # remove line-objects to autoscale axes (there is definetly a better way, but this works ¯\_(ツ)_/¯ )
    lo_00.remove()
    lo_01.remove()
    # lo_02.remove()
        # plot new pressure and velocity distribution in the pipeline
    lo_00, = axs1[0].plot(pl_vec,pressure_conversion(pipe.p_old,initial_pressure_unit, conversion_pressure_unit)[0],marker='.',c='blue')
    lo_01, = axs1[1].plot(pl_vec,pipe.v_old,marker='.',c='blue')
    # lo_02, = axs1[2].plot(level_vec_2,c='blue')
    fig1.suptitle(str(it_pipe))
    fig1.canvas.draw()
    fig1.tight_layout()
    plt.pause(0.00001)    

    # prepare for next loop
    p_old = pipe.p_old
    v_old = pipe.v_old  

        
        

In [6]:
# plot time evolution of boundary pressure and velocity as well as the reservoir level

fig2,axs2 = plt.subplots(3,2)
axs2[0,0].plot(t_vec,pressure_conversion(p_boundary_res,initial_pressure_unit, conversion_pressure_unit)[0])
axs2[0,1].plot(t_vec,v_boundary_res)
axs2[1,0].plot(t_vec,pressure_conversion(p_boundary_tur,initial_pressure_unit, conversion_pressure_unit)[0])
axs2[1,1].plot(t_vec,v_boundary_tur)
axs2[2,0].plot(t_vec,level_vec)
axs2[0,0].set_title('Pressure reservoir')
axs2[0,1].set_title('Velocity reservoir')
axs2[1,0].set_title('Pressure turbine')
axs2[1,1].set_title('Velocity turbine')
axs2[2,0].set_title('Level reservoir')
axs2[0,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[0,0].set_ylabel(r'$p$ ['+conversion_pressure_unit+']')
axs2[0,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[0,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
axs2[1,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[1,0].set_ylabel(r'$p$ ['+conversion_pressure_unit+']')
axs2[1,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[1,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
axs2[2,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[2,0].set_ylabel(r'$h$ [m]')
axs2[2,1].axis('off')
fig2.tight_layout()
plt.show()